# Implementando un Modelo de Churn de Clientes con PMML

### Instalamos los pre-requisitos y cargamos las librerías necesarias

In [ ]:
#pip install lightgbm nyoka

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from scipy import stats
from teradataml import create_context, DataFrame, get_context, copy_to_sql, in_schema, remove_context
import getpass as gp

In [ ]:
td_context = create_context(host="tdprd.td.teradata.com", username="lc250058", password=gp.getpass(prompt='Password:'), logmech="LDAP")

### Leemos el dataset de desarrollo

In [ ]:
# Create DataFrame for the tables in Vantage
churn_df = DataFrame("churn")
churn_df.shape

In [ ]:
# Using to_pandas() for a cleaner display format
desarrll = churn_df.to_pandas()
desarrll.head()

### Cambiamos los tipos de datos para que los acepte Python

In [ ]:
desarrll.DayMins = desarrll.DayMins.astype('float64')
desarrll.EveMins = desarrll.EveMins.astype('float64')
desarrll.NightMins = desarrll.NightMins.astype('float64')
desarrll.IntlMins = desarrll.IntlMins.astype('float64')
desarrll.DayCharge = desarrll.DayCharge.astype('float64')
desarrll.EveCharge = desarrll.EveCharge.astype('float64')
desarrll.NightCharge = desarrll.NightCharge.astype('float64')
desarrll.IntlCharge = desarrll.IntlCharge.astype('float64')

### Revisamos los valores nulos o missings

In [ ]:
desarrll.isnull().sum()

### División en train y test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(desarrll.drop(['Churn','State','AreaCode'],axis=1),
                                                   desarrll['Churn'],
                                                   test_size=0.33,
                                                   random_state=12345) 

### Creamos un pipeline para poder almacenar el Modelo LightGBM

In [ ]:
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier

pipeline_obj = Pipeline([('lgbmc',LGBMClassifier())])

pipeline_obj.fit(X_train,y_train)

### Exportamos el modelo generado utilizando la librería Nyoka

In [ ]:
from nyoka import lgb_to_pmml

lgb_to_pmml(pipeline_obj,desarrll.columns,'churn',"lgbmc_pmml.pmml")

### Ya podemos llevar el Modelo en formato PMML a Vantage

Copyright 2022 - Elaborado por Luis Cajachahua